In [1]:
import pandas as pd
import numpy as np
import os
import keras
#from tensorflow.keras import backend
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score

Using TensorFlow backend.


In [2]:
os.chdir(r'C:\Users\Vish\Documents\Data\Dataset')
df=pd.read_csv("sonar.csv")

In [3]:
df.head()

,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
0,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
1,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
2,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
3,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
4,0.0286,0.0453,0.0277,0.0174,0.0384,0.0990,0.1201,0.1833,0.2105,0.3039,...,0.0045,0.0014,0.0038,0.0013,0.0089,0.0057,0.0027,0.0051,0.0062,R


In [8]:
x=df.iloc[:5,0:60].values
y=df.iloc[:5,60].values

In [9]:
lb=LabelEncoder()
y=lb.fit_transform(y)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [ ]:
#Building model
#learn_rate and momentum are used if SGD optimizer is being use and need to be optimized separately 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV,cross_val_score
def build_classifier(units=1,init_mode='uniform', activation='relu',dropout_rate=0.0,optimizer='adam'):
    classifier=Sequential()
    classifier.add(Dense(units=units,kernel_initializer=init_mode,activation=activation,input_dim=60))
    classifier.add(Dense(units=units,kernel_initializer=init_mode,activation=activation))
    classifier.add(Dropout(dropout_rate))
    classifier.add(Dense(units=1,activation='sigmoid'))
    #optimizer = SGD(lr=learn_rate, momentum=momentum)
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier=KerasClassifier(build_fn=build_classifier)
                           
parameters ={'batch_size':[10,20],
             'epochs': [10,20],
             'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'],
             #'learn_rate' : [0.001, 0.01, 0.1],
             #'momentum' : [0.0, 0.2, 0.4],
             'init_mode' : ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'],
             'activation' : ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid'],
             'dropout_rate' : [0.0, 0.1, 0.2],
             'units':[30,32,34] 
            }

grid_search = GridSearchCV(estimator=classifier,
                           param_grid=parameters,
                           cv=3,
                           n_jobs=-1,  
                           return_train_score=True)

grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_
fit_time=grid_search.refit_time_
print(grid_search.best_estimator_)
print(best_parameters)
print(best_accuracy)
print(fit_time)

In [ ]:
# #Model Building
# Using Grid Search to find best parameters.
# learn_rate and momentum are used if SGD optimizer is being use and need to be optimized separately 
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Dropout
# from keras.wrappers.scikit_learn import KerasClassifier
# from sklearn.model_selection import GridSearchCV,cross_val_score
# def build_classifier(units=1,init_mode='uniform', activation='relu',dropout_rate=0.0,optimizer='adam'):
#     classifier=Sequential()
#     classifier.add(Dense(units=units,kernel_initializer=init_mode,activation=activation,input_dim=60))
#     classifier.add(Dense(units=units,kernel_initializer=init_mode,activation=activation))
#     classifier.add(Dropout(dropout_rate))
#     classifier.add(Dense(units=1,activation='linear'))
#     #optimizer = SGD(lr=learn_rate, momentum=momentum)
#     classifier.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['accuracy'])
#     return classifier

# classifier=KerasClassifier(build_fn=build_classifier)
                           
# parameters ={'batch_size':[10,20],
#              'epochs': [10,20],
#              'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'],
#              #'learn_rate' : [0.001, 0.01, 0.1],
#              #'momentum' : [0.0, 0.2, 0.4],
#              'init_mode' : ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'],
#              'activation' : ['relu'],
#              'dropout_rate' : [0.0, 0.1, 0.2],
#              'units':[30,32,34] 
#             }

# grid_search = GridSearchCV(estimator=classifier,
#                            param_grid=parameters,
#                            cv=3,
#                            n_jobs=-1,  
#                            return_train_score=True)

# grid_search = grid_search.fit(X_train, y_train)
# best_parameters = grid_search.best_params_
# best_accuracy = grid_search.best_score_
# fit_time=grid_search.refit_time_
# print(grid_search.best_estimator_)
# print(best_parameters)
# print(best_accuracy)
# print(fit_time)

In [ ]:
#Check how good our model with best parameters is
accuracies=cross_val_score(estimator=grid_search.best_estimator_,X=X_train,y=y_train,cv=10)
print(accuracies)
print(accuracies.mean())
print(accuracies.std())

In [ ]:
#Predictions
y_pred = grid_search.best_estimator_.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
#Evaluating model
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))